# Amapiano AI - MagnaTagATune Training Setup

**Purpose**: Train MusicGen on filtered MagnaTagATune dataset as Amapiano proxy

**Dataset**: 1,582 clips with electronic/jazzy/drum characteristics

**Duration**: 1-2 hours for quick test, 8-12 hours for full training

**Cost**: $0-10 (Colab Pro)

---

## Training Options
1. **Quick Test (1 epoch)**: Validate pipeline works (~1-2 hours)
2. **Short Training (5 epochs)**: Initial results (~4-6 hours)
3. **Full Training (20 epochs)**: Best results (~16-20 hours)

## Step 1: Mount Google Drive (for checkpoint persistence)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.makedirs('/content/drive/MyDrive/amapiano-training', exist_ok=True)
print("✅ Google Drive mounted successfully")

## Step 2: GPU Verification

In [ ]:
import torch

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print("\n✅ GPU AVAILABLE - Ready for training")
else:
    print("❌ GPU NOT AVAILABLE")
    print("⚠️  Go to Runtime > Change runtime type > Select GPU")
    raise RuntimeError("GPU required for training")

## Step 3: Clone Repository & Install Dependencies

In [ ]:
# Clone repo (replace with your GitHub URL if you've pushed it)
!git clone https://github.com/YOUR_USERNAME/amapiano-ai.git
%cd amapiano-ai

# Install dependencies
!pip install -q torch torchaudio transformers audiocraft datasets librosa pandas tqdm

print("✅ Dependencies installed")

## Step 4: Download & Filter MagnaTagATune Dataset

This creates the filtered dataset with ~1,582 clips matching Amapiano-proxy characteristics.

In [ ]:
%%time
import os
os.chdir('/content/amapiano-ai/ai-service')

# Download and filter the full dataset
!python dataset_setup.py \
  --output_dir ../datasets/amapiano_proxy \
  --min_score 3

import pandas as pd
from pathlib import Path

metadata_path = Path('../datasets/amapiano_proxy/training_metadata.csv')

if metadata_path.exists():
    df = pd.read_csv(metadata_path)
    print(f"\n✅ DATASET CREATED SUCCESSFULLY")
    print(f"   Total clips: {len(df)}")
    print(f"   Audio files: {len(list(Path('../datasets/amapiano_proxy/audio').glob('*.mp3')))}")
    print(f"   Total size: {sum(f.stat().st_size for f in Path('../datasets/amapiano_proxy/audio').glob('*.mp3')) / 1e9:.2f} GB")
    print(f"\nTop characteristics:")
    print(df['characteristics'].value_counts().head(10))
else:
    print("❌ DATASET NOT CREATED")
    raise RuntimeError("Dataset creation failed")

## Step 5: Choose Your Training Mode

Select ONE of the following cells to run based on your needs:

### Option A: Quick Pipeline Test (1 epoch, 100 samples)
**Duration**: ~15-30 minutes  
**Purpose**: Verify everything works before longer training

In [ ]:
%%time
!python train_musicgen.py \
  --data_dir ../datasets/amapiano_proxy/audio \
  --metadata ../datasets/amapiano_proxy/training_metadata.csv \
  --epochs 1 \
  --batch_size 1 \
  --max_samples 100 \
  --output_dir /content/drive/MyDrive/amapiano-training/quick-test

print("\n✅ Quick test complete! Check logs above for any errors.")

### Option B: Short Training (5 epochs, full dataset)
**Duration**: ~4-6 hours  
**Purpose**: Get initial trained model for evaluation

In [ ]:
%%time
!python train_musicgen.py \
  --data_dir ../datasets/amapiano_proxy/audio \
  --metadata ../datasets/amapiano_proxy/training_metadata.csv \
  --epochs 5 \
  --batch_size 2 \
  --output_dir /content/drive/MyDrive/amapiano-training/5-epoch

print("\n✅ 5-epoch training complete!")

### Option C: Full Training (20 epochs, full dataset)
**Duration**: ~16-20 hours  
**Purpose**: Best possible results with current dataset  
**Note**: Run overnight or use Colab Pro for longer sessions

In [ ]:
%%time
!python train_musicgen.py \
  --data_dir ../datasets/amapiano_proxy/audio \
  --metadata ../datasets/amapiano_proxy/training_metadata.csv \
  --epochs 20 \
  --batch_size 2 \
  --output_dir /content/drive/MyDrive/amapiano-training/20-epoch

print("\n✅ Full 20-epoch training complete!")

## Step 6: Verify Training Checkpoint

In [ ]:
from pathlib import Path

# Check which output directory you used
output_dirs = [
    "/content/drive/MyDrive/amapiano-training/quick-test",
    "/content/drive/MyDrive/amapiano-training/5-epoch",
    "/content/drive/MyDrive/amapiano-training/20-epoch"
]

for output_dir in output_dirs:
    checkpoint_dir = Path(output_dir)
    if checkpoint_dir.exists():
        checkpoint_files = list(checkpoint_dir.glob('*.pt')) + list(checkpoint_dir.glob('*.pth'))
        
        if checkpoint_files:
            print(f"\n✅ Checkpoints found in {output_dir}")
            for ckpt_file in checkpoint_files:
                print(f"   - {ckpt_file.name} ({ckpt_file.stat().st_size / 1e6:.1f} MB)")
            
            # Try to load the checkpoint
            try:
                ckpt = torch.load(checkpoint_files[0], map_location='cpu')
                print(f"\n   ✅ Checkpoint is loadable")
                print(f"   Keys: {list(ckpt.keys())[:5]}...")
                if 'epoch' in ckpt:
                    print(f"   Epoch: {ckpt['epoch']}")
                if 'loss' in ckpt:
                    print(f"   Loss: {ckpt['loss']:.4f}")
            except Exception as e:
                print(f"   ⚠️  Warning: Could not load checkpoint: {e}")
        else:
            print(f"   No checkpoints found in {output_dir}")

print("\n✅ Checkpoint verification complete")

## Step 7: Generate Test Sample

Generate a sample to hear the results (requires trained model)

In [ ]:
# TODO: Add generation code once train_musicgen.py includes generation capability
# For now, you'll need to load the model manually and generate

print("⚠️  Generation code to be added")
print("Your trained model is saved in Google Drive")
print("You can load it in the main application to test generation")

## Training Summary & Next Steps

In [ ]:
print("="*60)
print("TRAINING SUMMARY")
print("="*60)

print("\n📊 Dataset Statistics:")
if Path('../datasets/amapiano_proxy/training_metadata.csv').exists():
    df = pd.read_csv('../datasets/amapiano_proxy/training_metadata.csv')
    print(f"   Total clips: {len(df)}")
    print(f"   Avg score: {df['score'].mean():.2f}")
    print(f"   Duration: ~{len(df) * 29 / 3600:.1f} hours")

print("\n💾 Model Location:")
print("   /content/drive/MyDrive/amapiano-training/")

print("\n📝 Next Steps:")
print("   1. Review training logs above for loss trends")
print("   2. Load the checkpoint in your Amapiano AI app")
print("   3. Generate test samples and evaluate quality")
print("   4. Compare to baseline MusicGen (expect 15-25% authenticity)")

print("\n🎯 Expected Results:")
print("   - Better rhythm/beat patterns (from electronic/techno)")
print("   - Some piano elements (11.7% coverage)")
print("   - Improved from baseline but not authentic Amapiano yet")
print("   - Good foundation for further fine-tuning with real samples")

print("\n💰 Cost:")
print("   Colab Pro: ~$10/month (if used)")
print("   This training: 1-20 hours of GPU time")

print("\n" + "="*60)

---

## Troubleshooting

**GPU not available**: Runtime > Change runtime type > GPU  
**Out of memory**: Reduce `--batch_size` to 1  
**Session disconnected**: Checkpoints are saved in Google Drive, just rerun from Step 5  
**Loss is NaN**: Normal in first few steps, training will auto-recover  
**Download fails**: Check internet, retry dataset_setup.py

---

## Dataset Quality Notes

**Strengths:**
- 70.5% electronic/techno (good rhythmic foundation)
- 47.4% drums/beats (essential for Amapiano)
- 11.7% piano (some coverage)
- 10.4% jazzy (Private School style proxy)

**Limitations:**
- No authentic log drums (using bass as proxy)
- Low piano coverage for piano-heavy genre
- Genre mismatch (techno vs. house/deep house)
- Expected authenticity: 15-25% (vs 85-95% target)

**Recommendation:**
If results are promising (>20% authenticity, good rhythm), collect 500-1,000 real Amapiano tracks and retrain for 40-50% authenticity.

---

## References

- [MagnaTagATune Dataset](https://mirg.city.ac.uk/codeapps/the-magnatagatune-dataset)
- [MusicGen Paper](https://arxiv.org/abs/2306.05284)
- Project Docs: `/docs/INTERIM_DATASET_STRATEGY.md`